In [1]:
import ollama
import chromadb

In [2]:
# Base chat comletion
chat_response: ollama.ChatResponse = ollama.chat(model='llama3.2:1b', messages=[
  {
    'role': 'user',
    'content': 'What is llama?',
  },
])
print(chat_response['message']['content'])

Llama is a type of domesticated camelid, closely related to camels and alpacas. It belongs to the family Camelidae and is native to South America, specifically present in countries such as Peru, Bolivia, Chile, Argentina, and Ecuador.

Llamas are known for their distinctive appearance, with long necks, soft fur, and curved horns. They have a sturdy build and can weigh between 280 and 450 pounds (127 to 204 kilograms), depending on their breed and age.

There are several breeds of llama, including:

* Suri: Known for their soft, silky coats and distinctive "suris" or "horns."
* Huacaya: Characterized by their dense, woolly fleece.
* Alpaca: Similar to llamas but with a more compact body and softer hair.

Llamas are intelligent, social animals that can be used for packing, as pack animals, or as companions. They are also valued for their ability to help control weeds and brush on farms and pastures.

In some parts of the world, llamas are considered sacred animals, associated with good f

In [3]:
# Add document to db
documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

client = chromadb.Client()
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embed(model="mxbai-embed-large", input=d)
  embeddings = response["embeddings"]
  collection.add(
    ids=[str(i)],
    embeddings=embeddings,
    documents=[d]
  )

In [4]:
# get single query embedding
input_prompt = "What animals are llamas related to?"

# generate an embedding for the input and retrieve the most relevant doc
response = ollama.embed(
  model="mxbai-embed-large",
  input=input_prompt
)

In [5]:
# retrieval by search method
retrieval_count_limit = 3
results = collection.query(
  query_embeddings=response["embeddings"],
  n_results=retrieval_count_limit
)
print(results['documents'])

data = results['documents'][0][0]

[["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels", 'Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands', 'Llamas are vegetarians and have very efficient digestive systems']]


In [6]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama3.2:1b",
  prompt=f"Using this data: {data}. Respond to this prompt: {input_prompt}"
)

print(output['response'])

Llamas are indeed closely related to other camelids, which include:

1. Alpacas
2. Vicuñas
3. Camels (the same family as llamas)
4. Guanacos
5. Llama consorts and other relatives like the vicuña's wild relatives.

These animals share similar characteristics and are thought to have diverged from a common ancestor around 10,000-12,000 years ago during the Late Pleistocene era.
